# Creating Panel Dataset

#### Imports

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import phonetics as ph
import nltk
import random
from tqdm.notebook import tqdm
import statsmodels.api as sm
tqdm.pandas()
import os
import json
import ast
import shutil
import platform
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise_distances
import seaborn as sns
if platform.node() == 'Nick_Laptop':
    drive = 'C'
elif platform.node() == 'MSI':
    drive = 'D'
else:
    drive = 'uhhhhhh'
    print('Uhhhhhhhhhhhhh')
os.chdir(f'{drive}:/PhD/DissolutionProgramming/LND---Land-Paper')

PROCESSED = 'Data/Processed'
RAW = 'Data/Raw'
SURNAMES = f'{PROCESSED}/surname_info'


#### Loading

In [6]:
with open(f'{SURNAMES}/unique_id_dict.json', 'r') as f:
    unique_id_dict = json.load(f)
with open(f'{SURNAMES}/group_id_dict.json', 'r') as f:
    group_id_dict = json.load(f)
with open(f'{SURNAMES}/combined_id_dict.json', 'r') as f:
    combined_id_dict = json.load(f)
with open(f'{SURNAMES}/metaphone_id_dict.json', 'r') as f:
    metaphone_id_dict = json.load(f)
with open(f'{SURNAMES}/master_id_dict.json', 'r') as f:
    master_id_dict = json.load(f)


with open(f'{PROCESSED}/parish_correction.json', 'r') as j:
    parish_name_correction = json.loads(j.read())

mdf = pd.read_csv(f'{PROCESSED}/master_subsidy_data_final_with_parish_info.csv')
mdf['parish'] = mdf['parish'].apply(lambda x: parish_name_correction.get(x, x))

tdf = pd.read_csv(f'{PROCESSED}/tithe_landowners_final.csv')
tdf['parish'] = tdf['parish'].apply(lambda x: parish_name_correction.get(x, x))
tdf.rename(columns={'area_perches': 'value'})

wdf = pd.read_csv(f'{PROCESSED}/ukda_pcc_wills_final.csv')

cdf = pd.read_csv(f'{PROCESSED}/calendar_recipients_final.csv')

C:\Users\nicho\AppData\Local\Temp\ipykernel_13040\4219892269.py:16: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  mdf = pd.read_csv(f'{PROCESSED}/master_subsidy_data_final_with_parish_info.csv')


#### Creating Dictionary of First-Receipt Dates from Calendar

In [7]:
id_vars = ['unique_id', 'group_id', 'combined_id', 'metaphone_id', 'master_id']
receipt_date_dict = {}
for id_var in id_vars:
    receipt_dates = {}
    for id_no in cdf[id_var].unique():
        surname_df = cdf.loc[cdf[id_var] == id_no]
        earliest_year = surname_df['year'].min()
        receipt_dates[int(id_no)] = int(earliest_year)
    receipt_date_dict[id_var] = receipt_dates
with open(f'{PROCESSED}/receipt_date_dict.json', 'w') as j:
    j.write(json.dumps(receipt_date_dict, indent=4, sort_keys=True))

#### Putting in Lags and Leads for Individuals, Assigned Based on Surname

In [8]:
period_dict = {
    1524: 0,
    1543: 1,
    1581: 2,
    1674: 3,
    1840: 4
}

lags_1_dict = {
    1524:'lead_1',
    1543:'lag_1',
    1581:'lag_2',
    1674:'lag_3',
    1840:'lag_4'
}
lags_2_dict = {
    1524:'lead_2',
    1543:'lead_1',
    1581:'lag_1',
    1674:'lag_2',
    1840:'lag_3'
}

for id_var in id_vars:
    treatment_1_list = []
    treatment_2_list = []
    panel_df = mdf[[id_var, 'parish', 'year', 'value']].copy()
    panel_df = panel_df.rename(columns={id_var: 'id'})

    tithe_panel_df = tdf[[id_var, 'parish', 'area_perches']].copy()
    tithe_panel_df = tithe_panel_df.rename(columns={id_var: 'id', 'area_perches': 'value'})
    tithe_panel_df['year'] = 1840
    tithe_panel_df['parish'] = tithe_panel_df['parish'].apply(lambda x: parish_name_correction.get(x, x))
    panel_df = pd.concat([panel_df, tithe_panel_df], ignore_index=True)
    for k, v in receipt_date_dict[id_var].items():
        if v <= 1543:
            treatment_1_list.append(k)
        else:
            treatment_2_list.append(k)
    for i, row in tqdm(panel_df.iterrows(), total=panel_df.shape[0], desc=f'Processing {id_var} panel'):
        if row['id'] in treatment_1_list:
            panel_df.at[i, lags_1_dict[row['year']]] = 1
        elif row['id'] in treatment_2_list:
            panel_df.at[i, lags_2_dict[row['year']]] = 1
    for lag in lags_1_dict.values():
        panel_df.fillna({lag: 0}, inplace=True)
    for lag in lags_2_dict.values():
        panel_df.fillna({lag: 0}, inplace=True)
    panel_df['period'] = panel_df['year'].apply(lambda x: period_dict[x])
    panel_df.to_csv(f'{PROCESSED}/{id_var}_panel.csv', index=False)


    


Processing unique_id panel:   0%|          | 0/561730 [00:00<?, ?it/s]

Processing group_id panel:   0%|          | 0/561730 [00:00<?, ?it/s]

Processing combined_id panel:   0%|          | 0/561730 [00:00<?, ?it/s]

Processing metaphone_id panel:   0%|          | 0/561730 [00:00<?, ?it/s]

Processing master_id panel:   0%|          | 0/561730 [00:00<?, ?it/s]

#### Creating the Wills Panel

In [9]:
wdf['period'] = wdf['year'].apply(lambda x: (x - 1400) // 20)
dict_dict = {
    'unique_id': unique_id_dict,
    'group_id': group_id_dict,
    'combined_id': combined_id_dict,
    'metaphone_id': metaphone_id_dict,
    'master_id': master_id_dict
}
# Assign leads and lags based on the period in which each person in the calendar received their monastic land
for id_var in id_vars:
    id_type = id_var.split('_')[0]
    receipt_dates = receipt_date_dict[id_var]
    full_id_dict = dict_dict[id_var]
    # Get full list of ids
    full_ids = list(set(full_id_dict.values()))
    id_df = pd.DataFrame(full_ids, columns=['id'])
    id_df['receipt_year'] = id_df['id'].apply(lambda x: receipt_dates.get(x, np.nan))
    id_df['receipt_period'] = id_df['receipt_year'].apply(lambda x: (x - 1400) // 20 if not pd.isna(x) else x)
    # Need to make an entry for each id in each period
    panel_df = pd.DataFrame(columns=['id', 'period', 'receipt_period'])
    for period in range(0, 21):
        period_df = id_df.copy()
        period_df['period'] = period
        panel_df = pd.concat([panel_df, period_df], ignore_index=True)

    for i, row in tqdm(panel_df.iterrows(), total=panel_df.shape[0], desc=f'Processing {id_var} panel'):
        will_share = len(wdf.loc[(wdf[id_var] == row['id']) & (wdf['period'] == row['period'])])/len(wdf.loc[wdf['period'] == row['period']])
        panel_df.at[i, 'will_share'] = will_share        
        offset = row['receipt_period'] - row['period']
        if offset >= 0:
            panel_df.at[i, f'lead_{offset + 1}'] = 1
        else:
            panel_df.at[i, f'lag_{abs(offset)}'] = 1
    for col in panel_df.columns:
        if col.startswith('lead_') or col.startswith('lag_'):
            panel_df[col] = panel_df[col].fillna(0)
    panel_df.to_csv(f'{SURNAMES}/will_panel_{id_var}.csv', index=False)





C:\Users\nicho\AppData\Local\Temp\ipykernel_13040\2339091891.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  panel_df = pd.concat([panel_df, period_df], ignore_index=True)


Processing unique_id panel:   0%|          | 0/1070622 [00:00<?, ?it/s]

C:\Users\nicho\AppData\Local\Temp\ipykernel_13040\2339091891.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  panel_df = pd.concat([panel_df, period_df], ignore_index=True)


Processing group_id panel:   0%|          | 0/771792 [00:00<?, ?it/s]

C:\Users\nicho\AppData\Local\Temp\ipykernel_13040\2339091891.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  panel_df = pd.concat([panel_df, period_df], ignore_index=True)


Processing combined_id panel:   0%|          | 0/732543 [00:00<?, ?it/s]

C:\Users\nicho\AppData\Local\Temp\ipykernel_13040\2339091891.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  panel_df = pd.concat([panel_df, period_df], ignore_index=True)


Processing metaphone_id panel:   0%|          | 0/227367 [00:00<?, ?it/s]

C:\Users\nicho\AppData\Local\Temp\ipykernel_13040\2339091891.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  panel_df = pd.concat([panel_df, period_df], ignore_index=True)


Processing master_id panel:   0%|          | 0/255045 [00:00<?, ?it/s]